In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['PET']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [6]:
list(zip(longforms, counts))

[('positron emission tomography', 2317),
 ('polyethylene terephthalate', 112),
 ('poly ethylene terephthalate', 67),
 ('photoinduced electron transfer', 58),
 ('photosynthetic electron transport', 29),
 ('positron emission tomographic', 21),
 ('polyester', 12),
 ('pancreatic endocrine tumor', 11),
 ('positive emission tomography', 10),
 ('photo induced electron transfer', 10),
 ('peritoneal equilibration test', 6),
 ('primary endocrine therapy', 6),
 ('preemptive therapy', 6),
 ('paraffin embedded tissue', 5),
 ('paired end tag', 5),
 ('petalosa', 5),
 ('photosynthetic electron transfer', 4),
 ('patulous eustachian tube', 3),
 ('positron emission topography', 3),
 ('peptide exit tunnel', 3),
 ('positron emitting tomography', 3),
 ('positron emission computed tomography', 3),
 ('preeclampsia', 3)]

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts,
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'paired end tag': 'ungrounded',
  'pancreatic endocrine tumor': 'DOID:DOID:1799',
  'paraffin embedded tissue': 'ungrounded',
  'patulous eustachian tube': 'DOID:DOID:12358',
  'peptide exit tunnel': 'ungrounded',
  'peritoneal equilibration test': 'ungrounded',
  'petalosa': 'MESH:C506636',
  'photo induced electron transfer': 'GO:GO:0009767',
  'photoinduced electron transfer': 'GO:GO:0009767',
  'photosynthetic electron transfer': 'GO:GO:0009767',
  'photosynthetic electron transport': 'GO:GO:0009767',
  'poly ethylene terephthalate': 'CHEBI:CHEBI:61452',
  'polyester': 'MESH:D011091',
  'polyethylene terephthalate': 'CHEBI:CHEBI:61452',
  'positive emission tomography': 'MESH:D049268',
  'positron emission computed tomography': 'MESH:D049268',
  'positron emission tomographic': 'MESH:D049268',
  'positron emission tomography': 'MESH:D049268',
  'positron emission topography': 'MESH:D049268',
  'positron emitting tomography': 'MESH:D049268',
  'preeclampsia': 'MESH:D011225',
  'pr

In [11]:
grounding_map, names, pos_labels = [{'paired end tag': 'ungrounded',
  'pancreatic endocrine tumor': 'DOID:DOID:1799',
  'paraffin embedded tissue': 'ungrounded',
  'patulous eustachian tube': 'DOID:DOID:12358',
  'peptide exit tunnel': 'ungrounded',
  'peritoneal equilibration test': 'ungrounded',
  'petalosa': 'MESH:C506636',
  'photo induced electron transfer': 'GO:GO:0009767',
  'photoinduced electron transfer': 'GO:GO:0009767',
  'photosynthetic electron transfer': 'GO:GO:0009767',
  'photosynthetic electron transport': 'GO:GO:0009767',
  'poly ethylene terephthalate': 'CHEBI:CHEBI:61452',
  'polyester': 'MESH:D011091',
  'polyethylene terephthalate': 'CHEBI:CHEBI:61452',
  'positive emission tomography': 'MESH:D049268',
  'positron emission computed tomography': 'MESH:D049268',
  'positron emission tomographic': 'MESH:D049268',
  'positron emission tomography': 'MESH:D049268',
  'positron emission topography': 'MESH:D049268',
  'positron emitting tomography': 'MESH:D049268',
  'preeclampsia': 'MESH:D011225',
  'preemptive therapy': 'ungrounded',
  'primary endocrine therapy': 'ungrounded'},
 {'DOID:DOID:1799': 'islet cell tumor',
  'DOID:DOID:12358': 'patulous eustachian tube',
  'MESH:C506636': 'petalosa',
  'GO:GO:0009767': 'photosynthetic electron transport chain',
  'CHEBI:CHEBI:61452': 'poly(ethylene terephthalate) polymer',
  'MESH:D011091': 'Polyesters',
  'MESH:D049268': 'Positron-Emission Tomography',
  'MESH:D011225': 'Pre-Eclampsia'},
 ['CHEBI:CHEBI:61452', 'GO:GO:0009767']]

In [12]:
excluded_longforms = []

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:1848', 'HGNC:1848', 86)]

In [15]:
intersection2

[('MESH:D018893', 'HGNC:1848', 0),
 ('HGNC:1848', 'HGNC:1848', 1),
 ('ungrounded', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:64198', 'HGNC:1848', 0),
 ('MESH:D019164', 'HGNC:1848', 0),
 ('MESH:C059416', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:15710', 'HGNC:1848', 0)]

In [1]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

NameError: name 'unambiguous_agent_texts' is not defined

In [18]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 21:35:47] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 21:36:26] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9229216111746906 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'MESH:D049268': 1784,
  'CHEBI:CHEBI:61452': 127,
  'ungrounded': 20,
  'MESH:D011091': 8,
  'GO:GO:0009767': 66,
  'DOID:DOID:1799': 6,
  'DOID:DOID:12358': 2,
  'MESH:D011225': 2,
  'MESH:C506636': 1},
 'f1': {'mean': 0.922922, 'std': 0.027124},
 'precision': {'mean': 0.938945, 'std': 0.022214},
 'recall': {'mean': 0.911586, 'std': 0.049205},
 'DOID:DOID:12358': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'DOID:DOID:1799': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'GO:GO:0009767': {'f1': {'mean': 0.935897, 'std': 0.032179},
  'pr': {'mean': 0.894505, 'std': 0.060029},
  'rc': {'mean': 0.984615, 'std': 0.030769}},
 'ungrounded': {'f1': {'mean': 0.502857, 'std': 0.324056},
  'pr': {'mean': 0.4, 'std': 0.3},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'CHEBI:CHEBI:61452': {'f1': {'mean': 0.916235, 'std': 0.036001},
  'pr': {'mean': 0.920308, 'std

In [19]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [20]:
disamb.dump(model_name, results_path)

In [21]:
print(disamb.info())

Disambiguation model for PET

Produces the disambiguations:
	Polyesters	MESH:D011091
	Positron-Emission Tomography	MESH:D049268
	Pre-Eclampsia	MESH:D011225
	islet cell tumor	DOID:DOID:1799
	patulous eustachian tube	DOID:DOID:12358
	petalosa	MESH:C506636
	photosynthetic electron transport chain*	GO:GO:0009767
	poly(ethylene terephthalate) polymer*	CHEBI:CHEBI:61452

Class level metrics:
--------------------
Grounding                              	Count	F1     
           Positron-Emission Tomography	1784	0.98918
   poly(ethylene terephthalate) polymer*	 127	0.91624
photosynthetic electron transport chain*	  66	 0.9359
                             Ungrounded	  20	0.50286
                             Polyesters	   8	    0.0
                       islet cell tumor	   6	    0.0
               patulous eustachian tube	   2	    0.0
                          Pre-Eclampsia	   2	    0.0
                               petalosa	   1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.92292
	

In [22]:
model_to_s3(disamb)